In [ ]:
from typing import TypedDict, Optional
from langchain_aws import ChatBedrock
from langgraph.graph import StateGraph, END
import boto3
import json

# ==== AWS Configuration ====
REGION = "eu-west-2"
TARGET_LAMBDA = "test_function_26May2025"
KNOWLEDGE_BASE_ID = "ZTPJ2E7RTK"
MODEL_ARN = "arn:aws:bedrock:eu-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"

lambda_client = boto3.client("lambda", region_name=REGION)
brt_client = boto3.client("bedrock-agent-runtime", region_name=REGION)

# ==== LangGraph-compatible functions (nodes) ====

def lambda_summary_node(state):
    question = state["question"]
    schema_id = state.get("schema_id", "vitic_lhasa")

    response = lambda_client.invoke(
        FunctionName=TARGET_LAMBDA,
        InvocationType='RequestResponse',
        Payload=json.dumps({
            "question": question,
            "schema_id": schema_id
        })
    )
    payload = json.loads(response['Payload'].read())
    return {"question": question, "schema_id": schema_id, "narrative_response": payload.get("response", "")}

def bedrock_rag_node(state):
    question = state["question"]
    narrative_response = state["narrative_response"]

    combined_input = (
        f"[Structured Database Response]\n{narrative_response}\n\n"
        f"[Follow-up Question]\n{question}"
    )

    rag_response = brt_client.retrieve_and_generate(
        input={"text": combined_input},
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": KNOWLEDGE_BASE_ID,
                "modelArn": MODEL_ARN
            }
        }
    )
    return {"final_answer": rag_response["output"]["text"]}

# ==== Define the graph ====
class AgentState(TypedDict, total=False):
    question: str
    schema_id: Optional[str]
    narrative_response: Optional[str]
    final_answer: Optional[str]
    
graph = StateGraph(AgentState)
graph.add_node("lambda_summary", lambda_summary_node)
graph.add_node("bedrock_rag", bedrock_rag_node)

# Define transitions

graph.set_entry_point("lambda_summary")
graph.add_edge("lambda_summary", "bedrock_rag")
graph.set_finish_point("bedrock_rag")

# Compile the graph
app = graph.compile()


In [ ]:
inputs = {
    "question": "Is Omeprazole carcinogenic?",
    "schema_id": "vitic_lhasa"
}

result = app.invoke(inputs)
print("Final RAG response:\n", result["final_answer"])

Final RAG response:
 Based on the provided information, there is evidence that Omeprazole causes carcinogenic effects in rats, specifically increasing the incidence of malignant carcinoid tumors in the gastric mucosa. However, the carcinogenic potential in humans is unclear from the available data:

- In rats, Omeprazole exposure caused a statistically significant increase in malignant carcinoid tumors of the gastric mucosa, with higher potency observed in female rats compared to males (sources 1, 2).
- In mice, no statistically significant increase in tumor incidence was observed under the tested conditions (source 2).
- There is no direct data provided on carcinogenicity studies in humans. While the rat data suggests a carcinogenic potential for Omeprazole, the relevance to humans is uncertain. Some key points to consider:

- The mechanism of action leading to the rat tumors (increased gastrin levels causing enterochromaffin cell hyperplasia/neoplasia) does not appear to occur in humans receiving Omeprazole therapy based on clinical monitoring (source 3).
- Certain carcinogenic effects observed in rodents may not translate to humans due to differences in physiology and metabolism (source 3).
- Epidemiological studies in humans would be needed to definitively assess Omeprazole's carcinogenic risk in the clinical setting.